# GraphCast

## 1. 模型简介
Graphcast是谷歌Deepmind提出的一种用于全球中期天气预报的的方法，同时支持包括预测热带气旋路径、大气河流和极端气温在内的应用。该模型将地球天气的两个最新状态(当前时间和六个小时前)作为输入，并预测六小时后的天气状态。一个单独的天气状态由一个0.25维度/经度网格(721x1440)表示，这相当于赤道上大约28x28公里的分辨率，其中每个网格点代表一组地面和大气变量。本项目是利用torch框架对GraphCast的重构版本，主要用于模型训练。


**模型结构**

Graphcast是一种基于机器学习的天气预报大模型，性能优于世界上最准确的机器学习天气预报系统。Deepmind以编码-处理-解码的方式使用图神经网络(Graph Neural Network，GNN)来创建一个自回归模型。
Graphcast三阶段模拟过程如下：
1）第一阶段：编码阶段基于单个GNN将输入网格上表示为节点属性的变量映射到多网格图表示上的学习节点。
2）第二阶段：处理器阶段使用16个非共享GNN层来在多网格上执行学习信息传递，使得能够以很少的消息传递步骤进行有效的本地和远程信息传递。
3）第三阶段：解码器将来自多网格表示的第二阶段最终学习到的特征映射回纬度-经度网格，与第一阶段类似使用单个GNN层，并将输出预测作为残差更新最新的输入状态。

<img src=../../../doc/graphcast_architecture.png width=600 height=450 />



## 2. 软件环境准备

### 2.1 基于dtk适配软件
<div class="alert alert-warning"> WARNING：镜像中环境已配置，此步骤省略 </div>
<p>1. 基础软件环境DTK：推荐环境 dtk=dtk-24.04.2  下载链接：<a href="https://cancon.hpccube.com:65024/1/main/DTK-24.04.2/Ubuntu22.04" target="_blank">https://cancon.hpccube.com:65024/1/main/DTK-24.04.2/Ubuntu22.04 </a> </p>
<p>2. pytorch软件包下载：推荐环境 torch=2.1.0 py310 下载链接: <a href="https://download.sourcefind.cn:65024/4/main/pytorch/DAS1.2" target="_blank">https://download.sourcefind.cn:65024/4/main/pytorch/DAS1.2 </a> </p>
<p>3. torchvision软件包下载：推荐环境 torchvision=0.16.0 py310 下载链接: <a href="https://download.sourcefind.cn:65024/4/main/vision/DAS1.2" target="_blank">https://download.sourcefind.cn:65024/4/main/vision/DAS1.2 </a> </p>
<p>4. torchaudio软件包下载：推荐环境 torchaudio=2.1.2 py310 下载链接: <a href="https://download.sourcefind.cn:65024/4/main/

### 2.2 软件环境检查

In [ ]:
# 检查torch版本
import torch
import os
import onescience

version = torch.__version__
num = float(version[:3])
# assert num == 1.10

# 检查硬件环境
device = "cpu"
if os.system('rocm-smi 2>/dev/null || hy-smi 2>/dev/null')==0:
    device = "dtk"

elif os.system('nvidia-smi 2>/dev/null')==0: 
    device = "cuda"

print("torch version:", version)
print("onescience version:", onescience.__version__)
print("device =", device)

### 2.3 软件依赖安装


<div class="alert alert-warning"> WARNING: 镜像中环境已配置，此步骤省略 </div>

<div class="alert alert-note" style="color: blue;">
Note: 检查镜像环境下onescience版本号是否与当前目录onescience安装包版本一致，若不一致则需要卸载当前环境下的 onescience 包，并安装当前目录的 whl 包。安装指令参考：
<pre><code>pip uninstall onescience&&pip install &lt;onescience安装包名称&gt;.whl</code></pre>
</div>

## 3. 素材准备
### 3.1 数据集准备


请确保当前项目中包含dataset目录且结构如下
```
dataset\
|----train\
|    |----1980.h5
|    |----1981.h5
|    |----1982.h5
|    |----...
|----test\
|    |----2018.h5
|    |----2019.h5
|----stats\
|    |----global_means.npy
|    |----global_stds.npy
|----static\
|    |----geopotential.nc
|    |----land_sea_mask.nc
|----time_diff_std\
|    |----time_diff_std_new.npy
|    |----time_diff_mean_new.npy
|----data.json\
```
其中data.json中存储每个channel的名称，格式如下：

```
{
    "coords": {
        "channel": {
            "0": "t2m",
            "1": "msl",
            "2": "z50",
            "3": "q100"
            ...
        }
    }
}
```

主要目录结构如下
```
graphcast-torch\
|----conf\
|    |----config.yaml
|----loss\
|    |----__init__.py
|    |----util.py
|----onescience-0.1.0-py3-none-any.whl
|----README.md
|----graphcast-pytorch.ipynb
|----train_base.py
|----train_graphcast.py
|----train_utils.py
|----validation_base.py
```

## 4 训练
### 4.1 单卡训练
模型训练配置在./conf/config.yaml，主要参数介绍：
- processor_type：处理器类型，graphcast使用"MessagePassing"；
- hidden_dim: 层的维度size；
- latlon_res：输入分辨率，0.25°为[721,1440]；
- num_samples_per_year_train：每年的样本个数；
- num_channels_climate：气象变量数量，包括地表变量和大气变量；
- num_channels_static：静态变量个数；
- use_cos_zenith：是否使用太阳天顶角；
- amp：是否使用混合精度训练；
- use_wandb：是否使用`wandb`在线记录训练过程，默认`False`；
- save_freq：模型保存的频率；
- ckpt_path：模型保存地址。


In [ ]:
# 执行训练
!python train_graphcast.py

### 4.1 单节点多卡训练

In [ ]:
# 执行训练
!torchrun --standalone --nnodes=1 --nproc_per_node=4 train_graphcast.py

## 5. 相关文献和引用
https://github.com/google-deepmind/graphcast
